In [5]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='api_key', 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [68]:
from langchain_community.document_loaders import WebBaseLoader
from bs4 import BeautifulSoup
import re

# Load webpage
loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data = loader.load().pop().page_content

# Use BeautifulSoup to clean the HTML
soup = BeautifulSoup(page_data, "html.parser")

# Extract only visible text
clean_text = soup.get_text(separator=" ", strip=True)

# Remove extra spaces/newlines
clean_text = re.sub(r"\s+", " ", clean_text)

# Store it into variable as passage
page_data = clean_text  

print(page_data)  # print first 500 chars to check

Nike Careers Skip to main content Open Virtual Assistant Home Career Areas Total Rewards Life@Nike Purpose Language Select a Language Deutsch English Español (España) Español (América Latina) Français Italiano Nederlands Polski Tiếng Việt Türkçe 简体中文 繁體中文 עִברִית 한국어 日本語 Careers Close Menu Careers Chat Home Career Areas Total Rewards Life@Nike Purpose Jordan Careers Converse Careers Language Menu Return to Previous Menu Select a Language Deutsch English Español (España) Español (América Latina) Français Italiano Nederlands Polski Tiếng Việt Türkçe 简体中文 繁體中文 עִברִית 한국어 日本語 Your browser doesn't support HTML5 video tag. Play Full Video WHAT MOVES YOUMOVES OUR WORLD Pause Video Play Video Play 'How We Do It' video Play 'Victory Mode' video' Play 'Craft the Future' video Be In the Know The Nike Internship Experience Learn More AI/ML Roles for Innovators Learn More Peek Into Our World Learn More NikeUNITED's Commitment Learn More IF YOU HAVE A BODY, YOU ARE AN ATHLETE.* These words drive wh

In [74]:
page_data = """
Nike, Inc. is a global leader in the design, development, and marketing of athletic footwear, apparel, equipment, and accessories.
We are seeking a highly motivated and experienced Senior Software Engineer to join our dynamic team. The ideal candidate will have a strong background in software development, a passion for innovation, and a commitment to excellence."""

In [75]:
# pip install langchain-community


In [76]:
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [77]:
# pip install beautifulsoup4

In [78]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Software Engineer',
 'experience': 'experienced',
 'skills': ['software development', 'innovation'],
 'description': 'We are seeking a highly motivated and experienced Senior Software Engineer to join our dynamic team. The ideal candidate will have a strong background in software development, a passion for innovation, and a commitment to excellence.'}

In [79]:
type(json_res)

dict

In [80]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [81]:
import uuid
import chromadb

#persistent storage in vectorstore directory in the project folder
# if i used Client in taht case it will store in memory and will be lost once the program stops
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [82]:
# If json_res is a list of dicts, take the first dict
if isinstance(json_res, list) and len(json_res) > 0:
    json_res = json_res[0]

print(type(json_res))  # <class 'dict'>
print(json_res)  

<class 'dict'>
{'role': 'Senior Software Engineer', 'experience': 'experienced', 'skills': ['software development', 'innovation'], 'description': 'We are seeking a highly motivated and experienced Senior Software Engineer to join our dynamic team. The ideal candidate will have a strong background in software development, a passion for innovation, and a commitment to excellence.'}


In [83]:
page_data = ""

In [84]:
json_res

{'role': 'Senior Software Engineer',
 'experience': 'experienced',
 'skills': ['software development', 'innovation'],
 'description': 'We are seeking a highly motivated and experienced Senior Software Engineer to join our dynamic team. The ideal candidate will have a strong background in software development, a passion for innovation, and a commitment to excellence.'}

In [85]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}]]

In [87]:
job = json_res
job['skills']

['software development', 'innovation']

In [ ]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Amarendra, a business development executive at ak.tech is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of ak.tech 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase ak.tech portfolio: {link_list}
        Remember you are Mohan, BDE at ak.tech. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Development Solutions for Your Business

Dear Hiring Manager,

I came across your job posting for a Senior Software Engineer and was impressed by your company's commitment to innovation and excellence. As a Business Development Executive at ak.tech, I'd like to introduce you to our AI & Software Consulting company, which has a proven track record of empowering enterprises with tailored solutions that drive scalability, process optimization, cost reduction, and heightened overall efficiency.

Our team of expert software engineers has a strong background in software development and a passion for innovation, aligning perfectly with your job requirements. We've successfully delivered numerous projects that showcase our capabilities in software development, including our Kotlin Backend Portfolio (https://example.com/kotlin-backend-portfolio) and iOS AR Portfolio (https://example.com/ios-ar-portfolio). These projects demonstrate our ability to design, develop, and de